In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
from scipy.spatial import ConvexHull

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir3='/Volumes/My Passport/'
dir2='/Volumes/Extreme Pro/'

In [ ]:
ds=xr.open_dataset(dir2+'merra2_gridareas.nc')
areas=ds['cell_area']/1e6

yrs=np.arange(1980,2024,1)
area_list_fixed=[]
ivt_list_fixed=[]
iwv_list_fixed=[]
s_list_fixed=[]
mfc_list_fixed=[]

area_list_poleward=[]
ivt_list_poleward=[]
iwv_list_poleward=[]
s_list_poleward=[]
mfc_list_poleward=[]

area_list_variable=[]
ivt_list_variable=[]
iwv_list_variable=[]
s_list_variable=[]
mfc_list_variable=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)
        
    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')
        
    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values
        
    land_mask=dk['land']
    
    if 1==0:
        ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(year)+'.nc')
        ivt=ds['ivt'][:,:,:]
        ds=xr.open_dataset(dir2+'era_5_iwvinterp_'+str(year)+'.nc')
        iwv=ds['iwv'][:,:,:]
        ds=xr.open_dataset(dir2+'era_5_u850interp_'+str(year)+'.nc')
        u=ds['u']
        ds=xr.open_dataset(dir2+'era_5_v850interp_'+str(year)+'.nc')
        v=ds['v']
        s=np.sqrt(u**2+v**2)

        ds=xr.open_dataset(dir2+'era_5_mfcinterp_'+str(year)+'.nc')#era5 is divergence
        mfc=ds['mfc'][:,:,:]*-1.
    if 1==1:
        ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(year)+'.nc')
        ivt=ds['ivt'][0::2,:,:]
        ds=xr.open_dataset(dir2+'merra2.iwv.3hr.model.'+str(year)+'.nc')
        iwv=ds['iwv'][0::2,:,:]
        ds=xr.open_dataset(dir3+'merra2.u850.6hr.'+str(year)+'.nc')
        u=ds['u']
        ds=xr.open_dataset(dir3+'merra2.v850.6hr.'+str(year)+'.nc')
        v=ds['v']
        s=np.sqrt(u**2+v**2)
        
        ds=xr.open_dataset(dir2+'merra2.mfc.3hr.model.'+str(year)+'.nc')#merra-2 is tendency
        mfc=ds['mfc'][0::2,:,:]
        
    #ds=xr.open_dataset(dir2+'era5.ar.labels.model.poleward.'+str(year)+'.nc')
    #labeled=ds['ar_labeled']
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    #land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    #labeled=labeled.where(land_mask_interp>=0.5)
    ar=labeled/labeled
        
    ivt['lon']=labeled.lon.values
    iwv['lon']=labeled.lon.values
    s['lon']=labeled.lon.values
    mfc['lon']=labeled.lon.values
        
    ivt['lat']=labeled.lat.values
    iwv['lat']=labeled.lat.values
    s['lat']=labeled.lat.values
    mfc['lat']=labeled.lat.values
    
    iwv['time']=labeled.time.values
    s['time']=labeled.time.values
    
    print('A')
    
    year_length=365
    if year%4==0:
        year_length=366
        
    #dates=[dt.datetime(year,1,1,0)+dt.timedelta(hours=6*x) for x in range(4*year_length)]
    dates=[dt.datetime(year,1,1,0,30)+dt.timedelta(hours=6*x) for x in range(4*year_length)]
    area_year=0
    for j in range(len(dates)):
        ar_sel=ar.sel(time=dates[j])
        area=areas.where(ar_sel==1).sum().values.tolist()
        area_year=area_year+area
        
    area_list_poleward.append(area_year)

    print('B')
    if i==0:
        weights=np.cos(np.deg2rad(ds.lat))
        
    ivt_sel=ivt.where(labeled>0)
    ivt_weighted=ivt_sel.weighted(weights)
    ivt_mean=ivt_weighted.mean().values.tolist()
    ivt_list_poleward.append(ivt_mean)
    
    print(ivt_mean)
    print('C')
    iwv_sel=iwv.where(labeled>0)
    iwv_weighted=iwv_sel.weighted(weights)
    iwv_mean=iwv_weighted.mean().values.tolist()
    iwv_list_poleward.append(iwv_mean)
    
    print(iwv_mean)
    print('D')
    s_sel=s.where(labeled>0)
    s_weighted=s_sel.weighted(weights)
    s_mean=s_weighted.mean().values.tolist()
    s_list_poleward.append(s_mean)
    
    print(s_mean)
    print('E')
    mfc_sel=mfc.where(labeled>0)
    mfc_weighted=mfc_sel.weighted(weights)
    mfc_mean=mfc_weighted.mean().values.tolist()
    mfc_list_poleward.append(mfc_mean)
    print(mfc_mean)
    
    sys.exit()
    
    ###########
    
    #ds=xr.open_dataset(dir2+'era5.ar.labels.model.'+str(year)+'.nc')
    #labeled=ds['ar_labeled']
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled'][0::2,:,:]
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    ar=labeled/labeled
        
    ivt['lon']=labeled.lon.values
    iwv['lon']=labeled.lon.values
    s['lon']=labeled.lon.values
    mfc['lon']=labeled.lon.values
        
    ivt['lat']=labeled.lat.values
    iwv['lat']=labeled.lat.values
    s['lat']=labeled.lat.values
    mfc['lat']=labeled.lat.values
    
    iwv['time']=labeled.time.values
    s['time']=labeled.time.values
    
    print('A')
    
    area_year=0
    for j in range(len(dates)):
        ar_sel=ar.sel(time=dates[j])
        area=areas.where(ar_sel==1).sum().values.tolist()
        area_year=area_year+area
        
    area_list_fixed.append(area_year)

    print('B')
    if i==0:
        weights=np.cos(np.deg2rad(ds.lat))
        
    ivt_sel=ivt.where(labeled>0)
    ivt_weighted=ivt_sel.weighted(weights)
    ivt_mean=ivt_weighted.mean().values.tolist()
    ivt_list_fixed.append(ivt_mean)
    
    print('C')
    iwv_sel=iwv.where(labeled>0)
    iwv_weighted=iwv_sel.weighted(weights)
    iwv_mean=iwv_weighted.mean().values.tolist()
    iwv_list_fixed.append(iwv_mean)
    
    print('D')
    s_sel=s.where(labeled>0)
    s_weighted=s_sel.weighted(weights)
    s_mean=s_weighted.mean().values.tolist()
    s_list_fixed.append(s_mean)
    
    print('E')
    mfc_sel=mfc.where(labeled>0)
    mfc_weighted=mfc_sel.weighted(weights)
    mfc_mean=mfc_weighted.mean().values.tolist()
    mfc_list_fixed.append(mfc_mean)
    
    
    ########
    
    #ds=xr.open_dataset(dir2+'era5.ar.labels.model.variable.'+str(year)+'.nc')
    #labeled=ds['ar_labeled']
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    ar=labeled/labeled
        
    ivt['lon']=labeled.lon.values
    iwv['lon']=labeled.lon.values
    s['lon']=labeled.lon.values
    mfc['lon']=labeled.lon.values
        
    ivt['lat']=labeled.lat.values
    iwv['lat']=labeled.lat.values
    s['lat']=labeled.lat.values
    mfc['lat']=labeled.lat.values
    
    iwv['time']=labeled.time.values
    s['time']=labeled.time.values
    
    print('A')
    
    area_year=0
    for j in range(len(dates)):
        ar_sel=ar.sel(time=dates[j])
        area=areas.where(ar_sel==1).sum().values.tolist()
        area_year=area_year+area
        
    area_list_variable.append(area_year)

    print('B')
    if i==0:
        weights=np.cos(np.deg2rad(ds.lat))
        
    ivt_sel=ivt.where(labeled>0)
    ivt_weighted=ivt_sel.weighted(weights)
    ivt_mean=ivt_weighted.mean().values.tolist()
    ivt_list_variable.append(ivt_mean)
    
    print('C')
    iwv_sel=iwv.where(labeled>0)
    iwv_weighted=iwv_sel.weighted(weights)
    iwv_mean=iwv_weighted.mean().values.tolist()
    iwv_list_variable.append(iwv_mean)
    
    print('D')
    s_sel=s.where(labeled>0)
    s_weighted=s_sel.weighted(weights)
    s_mean=s_weighted.mean().values.tolist()
    s_list_variable.append(s_mean)
    
    print('E')
    mfc_sel=mfc.where(labeled>0)
    mfc_weighted=mfc_sel.weighted(weights)
    mfc_mean=mfc_weighted.mean().values.tolist()
    mfc_list_variable.append(mfc_mean)

In [ ]:
#print(iwv_list_fixed)

fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)
ax.plot(yrs,area_list_fixed)
plt.show()

In [ ]:
dk=xr.Dataset()
dk['area_list_merra2_poleward']=(('time'),area_list_poleward)
dk['ivt_list_merra2_poleward']=(('time'),ivt_list_poleward)
dk['iwv_list_merra2_poleward']=(('time'),iwv_list_poleward)
dk['s_list_merra2_poleward']=(('time'),s_list_poleward)
dk['mfc_list_merra2_poleward']=(('time'),mfc_list_poleward)

dk['area_list_merra2_fixed']=(('time'),area_list_fixed)
dk['ivt_list_merra2_fixed']=(('time'),ivt_list_fixed)
dk['iwv_list_merra2_fixed']=(('time'),iwv_list_fixed)
dk['s_list_merra2_fixed']=(('time'),s_list_fixed)
dk['mfc_list_merra2_fixed']=(('time'),mfc_list_fixed)

dk['area_list_merra2_variable']=(('time'),area_list_variable)
dk['ivt_list_merra2_variable']=(('time'),ivt_list_variable)
dk['iwv_list_merra2_variable']=(('time'),iwv_list_variable)
dk['s_list_merra2_variable']=(('time'),s_list_variable)
dk['mfc_list_merra2_variable']=(('time'),mfc_list_variable)

dk.coords['time']=yrs
dk['area_list_merra2_poleward'].attrs["units"]='km2'
dk['ivt_list_merra2_poleward'].attrs["units"]='kg*m-1*s-1'
dk['iwv_list_merra2_poleward'].attrs["units"]='kg*m-2'
dk['s_list_merra2_poleward'].attrs["units"]='m*s-1'
dk['mfc_list_merra2_poleward'].attrs["units"]='kg*m-2*s-1'

dk['area_list_merra2_fixed'].attrs["units"]='km2'
dk['ivt_list_merra2_fixed'].attrs["units"]='kg*m-1*s-1'
dk['iwv_list_merra2_fixed'].attrs["units"]='kg*m-2'
dk['s_list_merra2_fixed'].attrs["units"]='m*s-1'
dk['mfc_list_merra2_fixed'].attrs["units"]='kg*m-2*s-1'

dk['area_list_merra2_variable'].attrs["units"]='km2'
dk['ivt_list_merra2_variable'].attrs["units"]='kg*m-1*s-1'
dk['iwv_list_merra2_variable'].attrs["units"]='kg*m-2'
dk['s_list_merra2_variable'].attrs["units"]='m*s-1'
dk['mfc_list_merra2_variable'].attrs["units"]='kg*m-2*s-1'

try:
    os.remove(dir_data+'time_series_simple_merra2_land.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'time_series_simple_merra2_land.nc',mode='w',format='NETCDF4')

In [ ]:
yrs=np.arange(1958,2024,1)

ds=xr.open_dataset('/Users/ahenny/Downloads/jra55_gridareas_125.nc')
areas=ds['cell_area']/1e6
area_list_fixed=[]
ivt_list_fixed=[]
iwv_list_fixed=[]
s_list_fixed=[]
mfc_list_fixed=[]

area_list_poleward=[]
ivt_list_poleward=[]
iwv_list_poleward=[]
s_list_poleward=[]
mfc_list_poleward=[]

area_list_variable=[]
ivt_list_variable=[]
iwv_list_variable=[]
s_list_variable=[]
mfc_list_variable=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)
        
    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')
        
    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values
        
    land_mask=dk['land']
    
    ds=xr.open_dataset(dir3+'jra55_ivt_'+str(year)+'.nc')
    ivt=ds['ivt']
    ds=xr.open_dataset(dir3+'jra55_iwv_'+str(year)+'.nc')
    iwv=ds['iwv']
    ds=xr.open_dataset(dir3+'jra55_u850_'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir3+'jra55_v850_'+str(year)+'.nc')
    v=ds['v']
    s=np.sqrt(u**2+v**2)
    
    ivt=ivt.interp(lon=s.lon.values,lat=s.lat.values)
    iwv=iwv.interp(lon=s.lon.values,lat=s.lat.values)
    
        
    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    #land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    #labeled=labeled.where(land_mask_interp>=0.5)
    ar=labeled/labeled
    ar=ar.fillna(0)
    ar=ar.interp(lon=s.lon.values,lat=s.lat.values)
    ar=ar.where(ar>=0.5)
    ar=ar/ar
        
    ivt['lon']=ar.lon.values
    iwv['lon']=ar.lon.values
    s['lon']=ar.lon.values
        
    ivt['lat']=ar.lat.values
    iwv['lat']=ar.lat.values
    s['lat']=ar.lat.values
    
    iwv['time']=ar.time.values
    s['time']=ar.time.values
    
    print('A')
    
    year_length=365
    if year%4==0:
        year_length=366
        
    dates=[dt.datetime(year,1,1,0)+dt.timedelta(hours=6*x) for x in range(4*year_length)]
    #dates=[dt.datetime(year,1,1,0,30)+dt.timedelta(hours=6*x) for x in range(4*year_length)]
    area_year=0
    for j in range(len(dates)):
        ar_sel=ar.sel(time=dates[j])
        area=areas.where(ar_sel==1).sum().values.tolist()
        area_year=area_year+area
        
    area_list_poleward.append(area_year)

    print('B')
    if i==0:
        weights=np.cos(np.deg2rad(ar.lat))
        
    ivt_sel=ivt.where(ar==1)
    ivt_weighted=ivt_sel.weighted(weights)
    ivt_mean=ivt_weighted.mean().values.tolist()
    ivt_list_poleward.append(ivt_mean)
    
    print('C')
    iwv_sel=iwv.where(ar==1)
    iwv_weighted=iwv_sel.weighted(weights)
    iwv_mean=iwv_weighted.mean().values.tolist()
    iwv_list_poleward.append(iwv_mean)
    
    print('D')
    s_sel=s.where(ar==1)
    s_weighted=s_sel.weighted(weights)
    s_mean=s_weighted.mean().values.tolist()
    s_list_poleward.append(s_mean)
    
    
    ###########
    
    ds=xr.open_dataset(dir_data+'jra55.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    ar=labeled/labeled
    ar=ar.fillna(0)
    ar=ar.interp(lon=s.lon.values,lat=s.lat.values)
    ar=ar.where(ar>=0.5)
    ar=ar/ar
        
    ivt['lon']=ar.lon.values
    iwv['lon']=ar.lon.values
    s['lon']=ar.lon.values
        
    ivt['lat']=ar.lat.values
    iwv['lat']=ar.lat.values
    s['lat']=ar.lat.values
    
    iwv['time']=ar.time.values
    s['time']=ar.time.values
    
    print('A')
    
    area_year=0
    for j in range(len(dates)):
        ar_sel=ar.sel(time=dates[j])
        area=areas.where(ar_sel==1).sum().values.tolist()
        area_year=area_year+area
        
    area_list_fixed.append(area_year)

    print('B')
    if i==0:
        weights=np.cos(np.deg2rad(ar.lat))
        
    ivt_sel=ivt.where(ar==1)
    ivt_weighted=ivt_sel.weighted(weights)
    ivt_mean=ivt_weighted.mean().values.tolist()
    ivt_list_fixed.append(ivt_mean)
    
    print('C')
    iwv_sel=iwv.where(ar==1)
    iwv_weighted=iwv_sel.weighted(weights)
    iwv_mean=iwv_weighted.mean().values.tolist()
    iwv_list_fixed.append(iwv_mean)
    
    print('D')
    s_sel=s.where(ar==1)
    s_weighted=s_sel.weighted(weights)
    s_mean=s_weighted.mean().values.tolist()
    s_list_fixed.append(s_mean)
    
    ########
    
    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    labeled=labeled.where(land_mask_interp>=0.5)
    ar=labeled/labeled
    ar=ar.fillna(0)
    ar=ar.interp(lon=s.lon.values,lat=s.lat.values)
    ar=ar.where(ar>=0.5)
    ar=ar/ar
        
    ivt['lon']=ar.lon.values
    iwv['lon']=ar.lon.values
    s['lon']=ar.lon.values
        
    ivt['lat']=ar.lat.values
    iwv['lat']=ar.lat.values
    s['lat']=ar.lat.values
    
    iwv['time']=ar.time.values
    s['time']=ar.time.values
    
    print('A')
    
    area_year=0
    for j in range(len(dates)):
        ar_sel=ar.sel(time=dates[j])
        area=areas.where(ar_sel==1).sum().values.tolist()
        area_year=area_year+area
        
    area_list_variable.append(area_year)

    print('B')
    if i==0:
        weights=np.cos(np.deg2rad(ar.lat))
        
    ivt_sel=ivt.where(ar==1)
    ivt_weighted=ivt_sel.weighted(weights)
    ivt_mean=ivt_weighted.mean().values.tolist()
    ivt_list_variable.append(ivt_mean)
    
    print('C')
    iwv_sel=iwv.where(ar==1)
    iwv_weighted=iwv_sel.weighted(weights)
    iwv_mean=iwv_weighted.mean().values.tolist()
    iwv_list_variable.append(iwv_mean)
    
    print('D')
    s_sel=s.where(ar==1)
    s_weighted=s_sel.weighted(weights)
    s_mean=s_weighted.mean().values.tolist()
    s_list_variable.append(s_mean)

In [ ]:

ds=xr.open_dataset(dir_data+'time_series_simple_era5_land.nc')
test=[x/365./4./148326000. for x in ds['area_list_era5_fixed'].values.tolist()]

yrs=np.arange(1940,2024,1)
fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)
ax.plot(yrs,test)
plt.axhline(y=0.02)
plt.show()

r=st.linregress(yrs,test)
print(r)

In [ ]:
dk=xr.Dataset()
dk['area_list_jra55_poleward']=(('time'),area_list_poleward)
dk['ivt_list_jra55_poleward']=(('time'),ivt_list_poleward)
dk['iwv_list_jra55_poleward']=(('time'),iwv_list_poleward)
dk['s_list_jra55_poleward']=(('time'),s_list_poleward)

dk['area_list_jra55_fixed']=(('time'),area_list_fixed)
dk['ivt_list_jra55_fixed']=(('time'),ivt_list_fixed)
dk['iwv_list_jra55_fixed']=(('time'),iwv_list_fixed)
dk['s_list_jra55_fixed']=(('time'),s_list_fixed)

dk['area_list_jra55_variable']=(('time'),area_list_variable)
dk['ivt_list_jra55_variable']=(('time'),ivt_list_variable)
dk['iwv_list_jra55_variable']=(('time'),iwv_list_variable)
dk['s_list_jra55_variable']=(('time'),s_list_variable)

dk.coords['time']=yrs
dk['area_list_jra55_poleward'].attrs["units"]='km2'
dk['ivt_list_jra55_poleward'].attrs["units"]='kg*m-1*s-1'
dk['iwv_list_jra55_poleward'].attrs["units"]='kg*m-2'
dk['s_list_jra55_poleward'].attrs["units"]='m*s-1'

dk['area_list_jra55_fixed'].attrs["units"]='km2'
dk['ivt_list_jra55_fixed'].attrs["units"]='kg*m-1*s-1'
dk['iwv_list_jra55_fixed'].attrs["units"]='kg*m-2'
dk['s_list_jra55_fixed'].attrs["units"]='m*s-1'

dk['area_list_jra55_variable'].attrs["units"]='km2'
dk['ivt_list_jra55_variable'].attrs["units"]='kg*m-1*s-1'
dk['iwv_list_jra55_variable'].attrs["units"]='kg*m-2'
dk['s_list_jra55_variable'].attrs["units"]='m*s-1'

try:
    os.remove(dir_data+'time_series_simple_jra55_land.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'time_series_simple_jra55_land.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir2+'merra2_gridareas.nc')
areas=ds['cell_area']/1e6

yrs=np.arange(1940,2024,1)
area_list_fixed=[]
area_list_poleward=[]
area_list_variable=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    #ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
    #labeled=ds['ar_labeled']
    ar=labeled/labeled
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)

    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')

    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values

    land_mask=dk['land']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    
    ar=ar.where(land_mask_interp>=0.5)
    
    year_length=365
    if year%4==0:
        year_length=366
        
    dates=[dt.datetime(year,1,1,0)+dt.timedelta(hours=6*x) for x in range(4*year_length)]
    #dates=[dt.datetime(year,1,1,0,30)+dt.timedelta(hours=6*x) for x in range(4*year_length)]
    area_year=0
    for j in range(len(dates)):
        ar_sel=ar.sel(time=dates[j])
        area=areas.where(ar_sel==1).sum().values.tolist()
        area_year=area_year+area
        
    area_list_poleward.append(area_year)
    
    ###########
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    #ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(year)+'.nc')
    #labeled=ds['ar_labeled'][0::2,:,:]
    ar=labeled/labeled
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)

    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')

    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values

    land_mask=dk['land']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    
    area_year=0
    for j in range(len(dates)):
        ar_sel=ar.sel(time=dates[j])
        area=areas.where(ar_sel==1).sum().values.tolist()
        area_year=area_year+area
        
    area_list_fixed.append(area_year)
    
    ########
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    #ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(year)+'.nc')
    #labeled=ds['ar_labeled']
    ar=labeled/labeled
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)

    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')

    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values

    land_mask=dk['land']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    
    area_year=0
    for j in range(len(dates)):
        ar_sel=ar.sel(time=dates[j])
        area=areas.where(ar_sel==1).sum().values.tolist()
        area_year=area_year+area
        
    area_list_variable.append(area_year)

In [ ]:
dk=xr.Dataset()
dk['area_list_variable']=(('time'),area_list_variable)
dk['area_list_fixed']=(('time'),area_list_fixed)
dk['area_list_fixed_poleward']=(('time'),area_list_fixed_poleward)
dk.coords['time']=yrs

try:
    os.remove(dir_data+'artmip_redo_area_land_variants_era5.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'artmip_redo_area_land_variants_era5.nc',mode='w',format='NETCDF4')